<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/2_Taller_Regresion_Polinomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Regresión Polinómica, Subajuste, Sobreajuste y Regularización**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

- Samuel Aparicio
- Santiago Caicedo
- Ferney Reina

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Reg_Polin_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

30 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

## **Situación**

Una importante firma de inversión inmobiliaria te ha contratado como consultor de ciencia de datos. Su proceso actual de valoración de propiedades es lento y se basa en la intuición de unos pocos expertos. Quieren que desarrolles un modelo de machine learning para predecir el precio de venta de las viviendas (`SalePrice`) de forma más precisa y sistemática.

Te han entregado el dataset "Ames Housing", que contiene una gran cantidad de información sobre viviendas vendidas recientemente. Tu tarea es construir el mejor modelo posible, pero más importante aún, justificar por qué tu modelo es robusto y fiable, explicando cómo has manejado la complejidad y el riesgo de sobreajuste.

### **Ejercicio 1: Carga y Preparación Inicial de los Datos**

Primero, carguemos las librerías necesarias y el dataset. Nos enfocaremos en un subconjunto de las variables numéricas para mantener el taller manejable, pero el principio se aplica a todo el dataset.

In [1]:
# Importa las librerías necesarias
### TU CÓDIGO AQUÍ ###
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Importación de herramientas de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

Mejorar visualización de dataframes y gráficos

In [2]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

Cargar el dataset

In [3]:
# Usamos el dataset Ames Housing desde su fuente original (requiere internet)
url = 'http://jse.amstat.org/v19n3/decock/AmesHousing.txt'
df = pd.read_csv(url, sep='\t')
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.00,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.00,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.00,Unf,0.00,441.00,"1,080.00",GasA,Fa,Y,SBrkr,1656,0,0,1656,1.00,0.00,1,0,3,1,TA,7,Typ,2,Gd,Attchd,"1,960.00",Fin,2.00,528.00,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.00,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.00,TA,TA,CBlock,TA,TA,No,Rec,468.00,LwQ,144.00,270.00,882.00,GasA,TA,Y,SBrkr,896,0,0,896,0.00,0.00,1,0,2,1,TA,5,Typ,0,NaN,Attchd,"1,961.00",Unf,1.00,730.00,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.00,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.00,TA,TA,CBlock,TA,TA,No,ALQ,923.00,Unf,0.00,406.00,"1,329.00",GasA,TA,Y,SBrkr,1329,0,0,1329,0.00,0.00,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,"1,958.00",Unf,1.00,312.00,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.00,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.00,Gd,TA,CBlock,TA,TA,No,ALQ,"1,065.00",Unf,0.00,"1,045.00","2,110.00",GasA,Ex,Y,SBrkr,2110,0,0,2110,1.00,0.00,2,1,3,1,Ex,8,Typ,2,TA,Attchd,"1,968.00",Fin,2.00,522.00,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.00,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.00,TA,TA,PConc,Gd,TA,No,GLQ,791.00,Unf,0.00,137.00,928.00,GasA,Gd,Y,SBrkr,928,701,0,1629,0.00,0.00,2,1,3,1,TA,6,Typ,1,TA,Attchd,"1,997.00",Fin,2.00,482.00,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [4]:
# Para este taller, solo usaremos algunas columnas clave para simplificar.
df_sub = df[['Overall Qual', 'Gr Liv Area', 'SalePrice']].copy()
df_sub.columns = ['OverallQual', 'GrLivArea', 'SalePrice'] # Renombrar para facilidad
df_sub.head()

,OverallQual,GrLivArea,SalePrice
0,6,1656,215000
1,5,896,105000
2,6,1329,172000
3,7,2110,244000
4,5,1629,189900


**Explicación de las variables del dataset reducido**

1. SalePrice: Precio de Venta.

Esta es la variable objetivo. Es el precio final por el cual se vendió la propiedad, medido en dólares estadounidenses.

2. OverallQual: Calidad General.

Es una variable ordinal que califica la calidad general del material y el acabado de la casa. Es una de las variables predictoras (X) más importantes.

Escala: Va de 1 a 10.

10: Muy Excelente

9: Excelente

...

2: Pobre

1: Muy Pobre

3. GrLivArea: Área Habitable sobre el Nivel del Suelo.

Es una variable numérica que mide el total de metros cuadrados de área habitable que está por encima del nivel del suelo. No incluye el área del sótano. Es una de las variables predictoras (X) más fuertes, ya que, lógicamente, casas más grandes tienden a ser más caras.

In [5]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   OverallQual  2930 non-null   int64
 1   GrLivArea    2930 non-null   int64
 2   SalePrice    2930 non-null   int64
dtypes: int64(3)
memory usage: 68.8 KB


### **Ejercicio 2: Dividir el conjunto de datos**

El método `.info()` muestra que no hay nulos en nuestro subconjunto. ¡Perfecto! Ahora, definamos nuestras variables `X` (predictoras) e `y` (objetivo) y dividamos los datos.

In [6]:
# Definir X e y
### TU CÓDIGO AQUÍ ###
x = df_sub.drop('SalePrice', axis=1)
y = df_sub['SalePrice']


In [7]:
# Dividir en entrenamiento y prueba (70/30)
### TU CÓDIGO AQUÍ ###
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


### **Ejercicio 3: Modelo con Sobreajuste**

Vamos a crear un modelo muy complejo para ver qué tan mal puede generalizar.

**Crear un Modelo Polinómico de Grado 5**

Usa `Pipeline` para combinar `PolynomialFeatures` (grado 5), `StandardScaler` y `LinearRegression`. Entrénalo con los datos de entrenamiento.

In [9]:
# Crear el pipeline para el modelo polinómico
### TU CÓDIGO AQUÍ ###
pipeline_poly = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

pipeline_poly.fit(x_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=5, include_bias=False)),
                ('scaler', StandardScaler()), ('model', LinearRegression())])

In [10]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
y_train_pred_poly = pipeline_poly.predict(x_train)
y_test_pred_poly = pipeline_poly.predict(x_test)

In [13]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
rmse_train_poly = np.sqrt(mean_squared_error(y_train, y_train_pred_poly))
rmse_test_poly = np.sqrt(mean_squared_error(y_test, y_test_pred_poly))

print(f"RMSE en Entrenamiento (Poli-5): ${rmse_train_poly:,.2f} ")
print(f"RMSE en Prueba (Poli-5): ${rmse_test_poly:,.2f} ")

RMSE en Entrenamiento (Poli-5): $34,152.35 
RMSE en Prueba (Poli-5): $41,101.60 


**Pregunta:** ¿Qué indica la diferencia que observas entre el error de entrenamiento y el de prueba? ¿Le recomendarías este modelo a la firma inmobiliaria? ¿Por qué?

- El modelo está funcionando muy bien con los datos de entrenamiento (error bajo), pero tiene un rendimiento algo peor con los datos de prueba (error más alto). Este comportamiento sugiere que el modelo ha aprendido demasiado bien las características específicas del conjunto de entrenamiento, pero no generaliza tan bien para nuevos datos.
- No recomendaría este modelo tal como está, principalmente por el posible sobreajuste. Aunque el modelo parece ajustarse bien a los datos de entrenamiento, la diferencia entre el error de prueba y el de entrenamiento indica que podría no generalizar bien a datos nuevos, lo cual es crucial para hacer predicciones precisas en situaciones reales.

### **Ejercicio 4: Aplicar Regularización**

Ahora, vamos a "curar" el sobreajuste. Usaremos los mismos `PolynomialFeatures` de grado 5, pero cambiaremos el modelo de regresión.

**Implementar Regresión Ridge**

Copia el pipeline anterior, pero reemplaza `LinearRegression` con `Ridge(alpha=10)`. `alpha` es la fuerza de la penalización.

In [14]:
# Crear el pipeline para Ridge
### TU CÓDIGO AQUÍ ###
pipeline_ridge = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Ridge(alpha=10))
])


In [15]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
pipeline_ridge.fit(x_train, y_train)


Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=5, include_bias=False)),
                ('scaler', StandardScaler()), ('model', Ridge(alpha=10))])

In [18]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
y_train_pred_ridge = pipeline_ridge.predict(x_train)
y_test_pred_ridge = pipeline_ridge.predict(x_test)

rmse_train_ridge = np.sqrt(mean_squared_error(y_train, y_train_pred_ridge))
rmse_test_ridge = np.sqrt(mean_squared_error(y_test, y_test_pred_ridge))

print(f"RMSE Entrenamiento (Ridge): ${rmse_train_ridge:,.2f}")
print(f"RMSE Prueba (Poli-25): ${rmse_test_poly:,.2f} ")
print(f"RMSE Prueba (Ridge): ${rmse_test_ridge:,.2f} ")


RMSE Entrenamiento (Ridge): $35,204.89
RMSE Prueba (Poli-25): $41,101.60 
RMSE Prueba (Ridge): $35,206.63 


**Interpreta los resultados:**
- Error en entrenamiento: El modelo Ridge tiene un error en entrenamiento bastante cercano a los datos de prueba (RMSE de $35,204.89), lo que indica que el modelo no está sobreajustado. En comparación con el modelo polinómico de grado 25 (que tiene un error mucho menor en el entrenamiento), el modelo Ridge parece ser más robusto y menos complejo.

- Error en prueba: El RMSE del modelo Ridge en los datos de prueba es 35,206.63, lo que es casi idéntico al error de entrenamiento, lo que sugiere que el modelo está generalizando mejor en comparación con el modelo polinómico. Por otro lado, el modelo polinómico tenía un error mucho mayor en los datos de prueba ($41,101.60), lo que puede indicar que el modelo polinómico está sobreajustado.

**Implementar Regresión Lasso**

Haz lo mismo, pero ahora con `Lasso(alpha=500, max_iter=10000)`. Lasso necesita un `alpha` más grande (porque la penalización L1 es diferente) y a veces más `max_iter` para converger.

In [19]:
# Crear el pipeline para Lasso
### TU CÓDIGO AQUÍ ###
pipeline_lasso = Pipeline([
    ("poly_features", PolynomialFeatures(degree=5, include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", Lasso(alpha=500, max_iter=10000))
])

pipeline_lasso.fit(x_train, y_train)

Pipeline(steps=[('poly_features',
                 PolynomialFeatures(degree=5, include_bias=False)),
                ('scaler', StandardScaler()),
                ('model', Lasso(alpha=500, max_iter=10000))])

In [20]:
# Entrenar el modelo
### TU CÓDIGO AQUÍ ###
y_train_pred_lasso = pipeline_lasso.predict(x_train)
y_test_pred_lasso = pipeline_lasso.predict(x_test)

In [25]:
# Calcular el error (RMSE) en entrenamiento y prueba y realizar un print de estos
### TU CÓDIGO AQUÍ ###
rmse_train_lasso = np.sqrt(mean_squared_error(y_train, y_train_pred_lasso))
rmse_test_lasso = np.sqrt(mean_squared_error(y_test, y_test_pred_lasso))
print(f"RMSE Entrenamiento (Lasso): ${rmse_train_lasso:,.2f}")
print(f"RMSE Prueba (Ridge): ${rmse_test_ridge:,.2f} ")
print(f"RMSE Prueba (Lasso): ${rmse_test_lasso:,.2f}")

RMSE Entrenamiento (Lasso): $35,352.92
RMSE Prueba (Ridge): $35,206.63 
RMSE Prueba (Lasso): $35,447.71


**Interpreta los resultados:**
- Error en Entrenamiento (Lasso): El modelo Lasso tiene un error de 35,352.92, que es ligeramente mayor que el error del modelo Ridge $35,204.89 en el conjunto de entrenamiento, pero la diferencia no es tan significativa.

- Error en Prueba: El error de Lasso en el conjunto de prueba es 35,447.71, que es más alto que el error del modelo Ridge ($35,206.63). Esto sugiere que Lasso no está generalizando tan bien como Ridge, ya que su error en los datos de prueba es algo mayor.

**Selección de Variables con Lasso**

Una de las grandes ventajas de Lasso es que puede eliminar variables. Vamos a ver cuántas de las características polinómicas que creamos (ej. `OverallQual^2`, `GrLivArea^5`, `OverallQual * GrLivArea^4`, etc.) fueron eliminadas.

In [31]:
# Extraer los coeficientes del modelo Lasso
### TU CÓDIGO AQUÍ ###
coef_lasso = pipeline_lasso.named_steps['model'].coef_
print(coef_lasso)


[  1963.76821109      0.              0.          35444.97862341
      0.              0.          10560.00802868      0.
      0.              0.          21555.1038162       0.
      0.             -0.           9492.34283088      0.
      0.             -0.             -0.         -20625.70275837]


In [30]:
# Extraer los nombres de las características generadas
# Guíate por el siguiente código: feature_names = pipeline_lasso.named_steps['poly_features'].get_feature_names_out(X.columns)
### TU CÓDIGO AQUÍ ###
feature_names = pipeline_lasso.named_steps['poly_features'].get_feature_names_out(x.columns)
print(feature_names)


['OverallQual' 'GrLivArea' 'OverallQual^2' 'OverallQual GrLivArea'
 'GrLivArea^2' 'OverallQual^3' 'OverallQual^2 GrLivArea'
 'OverallQual GrLivArea^2' 'GrLivArea^3' 'OverallQual^4'
 'OverallQual^3 GrLivArea' 'OverallQual^2 GrLivArea^2'
 'OverallQual GrLivArea^3' 'GrLivArea^4' 'OverallQual^5'
 'OverallQual^4 GrLivArea' 'OverallQual^3 GrLivArea^2'
 'OverallQual^2 GrLivArea^3' 'OverallQual GrLivArea^4' 'GrLivArea^5']


In [29]:
# Contar cuántos coeficientes son exactamente cero
### TU CÓDIGO AQUÍ ###
num_zero_coefs = np.sum(coef_lasso == 0)
print(f"Número de características eliminadas por Lasso (coef = 0): {num_zero_coefs}")


Número de características eliminadas por Lasso (coef = 0): 14


In [34]:
# Realizar un print de:
# Número total de características polinómicas generadas
# Número de características eliminadas por Lasso (coef = 0)
# Número de características CONSERVADAS
# Porcentaje de variables eliminadas
# Lista con los nombres de las características conservadas por Lasso (coef != 0)

### TU CÓDIGO AQUÍ ###
num_total_features = len(feature_names)
print(f"Número total de características polinómicas generadas: {num_total_features}")
print(f"Número de características eliminadas por Lasso (coef = 0): {num_zero_coefs}")
print(f"Número de características conservadas por Lasso (coef != 0): {num_total_features - num_zero_coefs}")
print(f"Porcentaje de variables eliminadas: {(num_zero_coefs / num_total_features) * 100:.2f}%")
print("Nombres de características conservadas:")
print(feature_names[coef_lasso != 0])


Número total de características polinómicas generadas: 20
Número de características eliminadas por Lasso (coef = 0): 14
Número de características conservadas por Lasso (coef != 0): 6
Porcentaje de variables eliminadas: 70.00%
Nombres de características conservadas:
['OverallQual' 'OverallQual GrLivArea' 'OverallQual^2 GrLivArea'
 'OverallQual^3 GrLivArea' 'OverallQual^5' 'GrLivArea^5']


**Interpreta los resultados:**
- El modelo generó un total de 20 características polinómicas a partir de las variables originales.
Tras aplicar la regularización Lasso, se eliminaron 14 características (aquellas con coeficiente igual a 0), lo que representa un 70% del total. Esto significa que solo el 30% de las variables fueron realmente útiles para explicar la variable objetivo.
- Esto indica que las variables OverallQual (calidad general de la vivienda) y GrLivArea (área habitable) —incluyendo sus combinaciones y potencias polinómicas— son las que aportan mayor poder explicativo al modelo.

- El modelo Lasso ayudó a reducir la complejidad eliminando el 70% de las -
variables irrelevantes, quedándose solo con las más influyentes, lo cual mejora la interpretabilidad y evita el sobreajuste.

### **Ejercicio 5: Conclusión y Recomendación para el Cliente**

**Resumir los Resultados**

Crea un `DataFrame` de pandas que compare el RMSE de entrenamiento y prueba de los tres modelos (Polinómico, Ridge, Lasso) y ordena los modelos según el RMSE de prueba de menor a mayor.

In [37]:
# Crear DataFrame de resultados
### TU CÓDIGO AQUÍ ###
results = pd.DataFrame({
    'Modelo': [
        '1. Polinómico Grado 5 ',
        '2. Ridge ',
        '3. Lasso '
    ],
    'RMSE en Entrenamiento': [
        rmse_train_poly,
        rmse_train_ridge,
        rmse_train_lasso
    ],
    'RMSE en Prueba': [
        rmse_test_poly,
        rmse_test_ridge,
        rmse_test_lasso
    ]
})

results_sorted = results.sort_values(by='RMSE en Prueba')

print("Tabla Comparativa de Rendimiento (Error en Dólares)")
display(results_sorted.style.format({
    'RMSE en Entrenamiento': '${:,.2f}',
    'RMSE en Prueba': '${:,.2f}'
}).background_gradient(cmap='Reds', subset=['RMSE en Prueba']))

Tabla Comparativa de Rendimiento (Error en Dólares)


,Modelo,RMSE en Entrenamiento,RMSE en Prueba
1,2. Ridge,"$35,204.89","$35,206.63"
2,3. Lasso,"$35,352.92","$35,447.71"
0,1. Polinómico Grado 5,"$34,152.35","$41,101.60"


**Pregunta Final:**

Basado en tu análisis, ¿qué modelo le recomendarías a la firma inmobiliaria? Tu respuesta debe incluir:

1.  Una recomendación clara del mejor modelo.
2.  Una explicación en términos sencillos (para un gerente, no para un científico de datos) de por qué el modelo polinómico simple no era una buena idea, usando el concepto de "memorizar vs. generalizar".
3.  Una descripción de la ventaja principal del modelo Lasso en cuanto a la interpretabilidad y la selección de las características más importantes.

## Solución
1. Recomendaría a la firma inmobiliaria utilizar el modelo Ridge, ya que presenta el menor error en prueba (RMSE = $35,206.63), lo que indica que logra predecir los precios de manera más precisa con datos nuevos. Además, su desempeño en entrenamiento y prueba es muy similar, lo que muestra que generaliza bien y no depende excesivamente de los datos con los que fue entrenado.
2. El modelo polinómico simple de grado 5 tuvo el menor error durante el entrenamiento, pero un error muy alto en prueba. Esto significa que “memorizó” los datos en lugar de “aprender patrones generales”, un problema conocido como sobreajuste. En otras palabras, el modelo se adaptó demasiado a los ejemplos que ya conocía, pero no logra predecir bien cuando se enfrenta a nuevos casos.
3. Aunque el modelo Ridge fue el más preciso, el modelo Lasso tiene una ventaja importante: permite identificar las variables más relevantes, eliminando las menos útiles. Esto mejora la interpretabilidad del modelo, ayudando a la empresa a entender qué factores influyen más en el precio de las viviendas. En este caso, Lasso redujo el número de características en un 70%, quedándose con variables clave como OverallQual (calidad general) y GrLivArea (área habitable), lo que facilita la toma de decisiones basada en información clara y relevante.